In [6]:
# imports
import keras
import numpy as np
import pandas as pd
import torch_cnn as tcnn
from keras_cnn import keras_CNN as kcnn
import data_processing as d
from tensorflow import keras
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split   

In [12]:
# Data retieval
X, y = d.get_encoded_data()
X_train, X_test, y_train, y_test = d.splitData(X, y)

# Keras CNN Data
# X_train = X_train.reshape(1, X_train.shape[0], X_train.shape[1])
# y_train = y_train.reshape(1, y_train.shape[0], y_train.shape[1])
# X_test = X_test.reshape(1, X_test.shape[0], X_test.shape[1])
# y_test = y_test.reshape(1, y_test.shape[0], y_test.shape[1])
    
# X_train = X_train.astype(np.float32)
# y_train = y_train.astype(np.float32)


# Convert data to Tensors to Incease Computation time for Torch CNN
X_tensor_train,Y_tensor_train  = d.toTensors(X, y)

# Change Y data for SVM
svm_X_train, svm_X_test, svm_y_train, svm_y_test =  train_test_split(X, d.get_SVM_y(), test_size=0.33, random_state=1)

In [8]:
# Variables

EPOCHS = 1000 # arbitrary

# Keras CNN
KERNEL_SIZE = 3
FILTERS = 128
DROPOUT_RATE = 0.25

# Torch CNN
N_FEATURES = X_train.shape[1] # Number of features for the input layer
NUM_ROWS_TRAINING = X_train.shape[0] # Number of rows
N_HIDDEN = N_FEATURES * 10 # Size of first linear layer
N_CNN_KERNEL, MAX_POOL_KERNEL = 3, 4 # CNN kernel size

In [14]:
# Keras CNN
#Build Model
cnn1D = kcnn(7, 500)
new_X_train, new_X_test, new_y_train, new_y_test = cnn1D.reshape_and_chng_type(X_train, X_test, y_train, y_test)
model = cnn1D.getModel()
optimizer = keras.optimizers.Nadam(learning_rate=1e-3, decay=5e-4)

model.compile(optimizer, 'mse', metrics=[keras.metrics.Accuracy()])

history = model.fit(
    new_X_train,
    new_y_train,
    batch_size=256,
    verbose=1,
    epochs=150
)

# for metric in history.history:
#     cnn1D.plot(history.history, EPOCHS, metric)

print("Evaluate on test data")
results = model.predict(new_X_test, batch_size=256)
results = results.reshape(results.shape[1], results.shape[2]).astype(int)
count = failures = successes = 0
for i in range(len(results)):
    for j in range(len(results[i])):
        count += 1
        if results[i][j] == int(y_test[i][j]):
            successes += 1

print("test acc:", float(successes) / count)

Epoch 1/150
1/1 [==============================] - 1s 1s/step - loss: 0.1448 - accuracy: 0.8161
Epoch 2/150
1/1 [==============================] - 0s 82ms/step - loss: 0.1320 - accuracy: 0.8295
Epoch 3/150
1/1 [==============================] - 0s 86ms/step - loss: 0.1146 - accuracy: 0.8527
Epoch 4/150
1/1 [==============================] - 0s 93ms/step - loss: 0.0772 - accuracy: 0.9069
Epoch 5/150
1/1 [==============================] - 0s 88ms/step - loss: 0.0640 - accuracy: 0.9286
Epoch 6/150
1/1 [==============================] - 0s 82ms/step - loss: 0.0595 - accuracy: 0.9370
Epoch 7/150
1/1 [==============================] - 0s 84ms/step - loss: 0.0580 - accuracy: 0.9398
Epoch 8/150
1/1 [==============================] - 0s 80ms/step - loss: 0.0580 - accuracy: 0.9395
Epoch 9/150
1/1 [==============================] - 0s 77ms/step - loss: 0.0569 - accuracy: 0.9416
Epoch 10/150
1/1 [==============================] - 0s 90ms/step - loss: 0.0569 - accuracy: 0.9423
Epoch 11/150
1/1 [===

In [ ]:
### Torch CNN
# Build CNN
net = tcnn.CNN(n_feature=N_FEATURES, n_hidden=N_HIDDEN, n_output=13, n_cnn_kernel=N_CNN_KERNEL)   # define the network    
# Train and Test
train_results, results = tcnn.trainTestCNN(net,X_tensor_train,Y_tensor_train, EPOCHS)
# Plotting
d.plot(results, 10, "Accuracy")

In [ ]:
# SVM
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
accuracy = []
# Train and Test
for x in range(0,EPOCHS):
    clf.fit(svm_X_train,  svm_y_train)
    if x % 100 == 0:
        predictions=clf.predict(svm_X_test)
        accuracy.append(accuracy_score(svm_y_test, predictions))
# Plotting        
results = {'Accuracy':accuracy}
for metric in results:
    kcnn.plot(results, EPOCHS/100, metric, "Testing")        

In [ ]:
df = pd.read_csv('datasets/VirusSample.csv')
df = df.drop(columns=['file'])
pd.set_option('max_colwidth', 150)
shuffle(df).head(20)